In [2]:
from legisph.senate.website import SenateWebsite
from legisph.senate.models import SenateBill

senate = SenateWebsite()
bills = list(filter(lambda b: not isinstance(b, Exception), senate.fetch_all_bills()))


In [3]:
import polars as pl
from itertools import chain

all_statuses = [
    *[b.legislative_status for b in bills],
    *list(chain(*[b.legislative_history for b in bills])),
]
all_statuses = list(
    chain(
        *[
            [
                SenateBill.SenateBillStatus(date=status.date, item=s)
                for s in status.item.split(";\n")
            ]
            for status in all_statuses
        ]
    )
)

In [5]:
from legisph.senate.bill_status import *

unparsed_statuses = [
    status
    for status in all_statuses
    if len(
        parse_senate_bill_status(
            status,
            classes=[
                Introduced,
                FirstReading,
                OrganizationalBriefing,
                PendingInCommittee,
                CommitteeReportCalendaredForOrdinaryBusiness,
                ConsolidatedOrSubstitutedInCommitteeReport,
                TechnicalWorkingGroup,
                CommitteeProceedings,
                SubmittedCommitteeReport,
                PendingSecondReading,
                ApprovedOnSecondReading,
                TransferredToCalendarForSpecialOrder,
                PeriodOfInterpellationClosed,
                SponsorshipSpeech,
                SponsorshipSpeechInsertedIntoRecord,
                ApprovedOnThirdReading,
                SentToHouseOfRepresentatives,
                IndividualAmendmentsOpened,
                IndividualAmendmentsClosed,
                CommitteeAmendmentsOpened,
                CommitteeAmendmentsClosed,
                CopiesDistributed,
                ConsolidatedWithApprovedBill,
                ApprovedByPresident,
                ApprovedConferenceCommitteeReport,
                Interpellation,
                PendingInHouseOfRepresentatives,
                Withdrawn,
                Archived,
                Sponsored,
                VotesInFavor,
                VotesAgainst,
                VotesAbstained,
                LapsedIntoLaw,
                InquiryOfChair
            ],
        )
    )
    == 0
]
status_df = pl.DataFrame({"status": [status.item for status in unparsed_statuses]})
remaining = list(
    (
        status_df.groupby("status")
        .agg([pl.count()])
        .sort("count", reverse=True)
        .filter(pl.col("count") > 10)
    )["status"]
)
# (
#     status_df.groupby("status")
#             .agg([pl.count()])
#             .sort("count", reverse=True)
#             .filter(pl.col("count") > 10)
# )
print(f"{len(remaining)}/{len(all_statuses)}")
remaining

63/93664


['Motion of the Majority Leader, Senator Alan Peter Compañero S. Cayetano, that the matters appearing in the Reference of Business for todays session be deemed read and transmitted to the Archives;',
 'Transferred from the Committee on Constitutional Amendments, Revision of Codes and Laws to the Committee on ELECTORAL REFORMS  AND PEOPLES PARTICIPATION;',
 'Change of committee referral from the Committee on Youth, Women and Family Relations to the Committee on WOMEN, FAMILY RELATIONS AND GENDER EQUALITY;',
 'Conducted CONSULTATION/OCULAR;',
 'Returned and submitted by the Committee on WAYS AND MEANS per Committee Report No. 164, recommending that it be substituted by SBN-1592;',
 'Conference Committee Report approved by the Senate;',
 'Sponsors: Senator(s) FRANCIS "CHIZ" G. ESCUDERO, EDGARDO J. ANGARA;',
 'Senator MANUEL "LITO" M. LAPID was made as coauthor;',
 'Reconsideration of Approval on Second Reading;',
 'Manifestation of Senator SOTTO III, as agreed upon in caucus, that bills f

In [34]:
from legisph.senate.bill_status import *
from legisph.senate.models import SenateBill
import datetime

matches = set([s.item for s in all_statuses if s.item.lower().find("\n") > -1])
matches
# unparsed_statuses = [
#     status
#     for status in sponsorship_speech
#     if SponsorshipSpeech.parse(SenateBill.SenateBillStatus(
#         date = datetime.date.today(),
#         item = status
#     ))[0] is None
# ]
# unparsed_statuses


{'\nEdit\nDelete',
 'Acknowledging Senator Drilon observation, upon motion of Senator Sotto III, SBN-1374 was referred primarily to the\nCommittee on PUBLIC ORDER AND DANGEROUS DRUGS and secondarily to the Committees on JUSTICE AND HUMAN RIGHTS; and Committee on FINANCE;',
 'Acting on the request of Senate President Pimentel, Senator Sotto III moved to change the referral\nof SBN-110 to the Committee on LOCAL GOVERNMENT as the primary committee and to the Committee on WAYS AND MEANS as the secondary committee;',
 'Acting on the request of Senator Gatchalian, upon motion of Senator Sotto III, the Body approved the transfer of the primary referral of SBN-1371 from the Committee on Public Services to the Committee on ENERGY, with\nthe Committees on Public Services, Ways and Means\nand Finance as the secondary committees;',
 'Against: None\nAbstention: None',
 'All Senators present were made  coauthors as manifested by Senator Cayetano (A.):\nSenators JUAN EDGARDO "SONNY" M. ANGARA, PAOLO 

In [53]:
a = list(
    filter(
        lambda b: "Consolidated/Substituted in the Committee Report"
        == b.legislative_status.item,
        bills,
    )
)
a[0].legislative_history


[SenateBillStatus(date=datetime.date(2022, 8, 31), item='Introduced by Senator CYNTHIA A. VILLAR;'),
 SenateBillStatus(date=datetime.date(2022, 9, 7), item='Read on First Reading and Referred to the Committees on PUBLIC SERVICES and TRADE, COMMERCE AND ENTREPRENEURSHIP;'),
 SenateBillStatus(date=datetime.date(2022, 9, 13), item='Returned and submitted jointly by the Committees on PUBLIC SERVICES, TRADE, COMMERCE AND ENTREPRENEURSHIP, SCIENCE AND TECHNOLOGY and BANKS, FINANCIAL INSTITUTIONS AND CURRENCIES per Committee Report No. 5, recommending that it be substituted by SBN-1310;'),
 SenateBillStatus(date=datetime.date(2022, 9, 13), item='Committee Report Calendared for Ordinary Business;'),
 SenateBillStatus(date=datetime.date(2022, 9, 13), item='SUBSTITUTED BY SBN-1310 UNDER COMMITTEE REPORT NO. 5.')]

In [56]:
a = list(
    filter(
        lambda b: "Conducted CONSULTATION/OCULAR;"
        in [h.item for h in b.legislative_history],
        bills,
    )
)


In [57]:
a[0]


SenateBill(
    url="http://legacy.senate.gov.ph/lis/bill_res.aspx?q=SBN-958&congress=18",
    congress=18,
    billno="SBN-958",
    congress_text="18th Congress",
    billno_text="Senate Bill No. 958",
    title="LOWERING THE OPTIONAL RETIREMENT AGE OF PUBLIC SCHOOL TEACHERS",
    long_title="AN ACT LOWERING THE OPTIONAL RETIREMENT AGE OF PUBLIC SCHOOL TEACHERS FROM SIXTY (60) YEARS OLD TO FIFTY-FIVE (55) YEARS OLD, AMENDING FOR THE PURPOSE SECTION 13-A OF REPUBLIC ACT NO. 8291, OTHERWISE KNOWN AS THE 'GOVERNMENT SERVICE INSURANCE SYSTEM ACT OF 1997'",
    filing_date=datetime.date(2019, 8, 27),
    filers=[Senator(name="Angara, Sonny")],
    links=[
        Link(
            url="http://legacy.senate.gov.ph/lisdata/3136128264!.pdf",
            name="SBN-958 (as filed)",
        )
    ],
    scope="National",
    legislative_status=SenateBillStatus(
        date=datetime.date(2019, 8, 28), item="Pending in the Committee"
    ),
    subjects=[
        Subject(name="Retirement Age"),


Cases to be handled: 
* items with newlines (\n) in them
* items with joint meanings but no newlines 
* senators names parsed correctly